# DATA612 - Assignment # 1
# BY: Sang Yoon (Andy) Hwang
# DATE: 2019-06-11

In [388]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split

# Description of recommender system
This system recommends movies to people based on movie ratings (1 to 5).

In [389]:
# Just create dataset with this format --> user, movie_title, rating and then use pivot/melt to create matrix
data = {'user':['Tom','Sally','Vincio','Andy','Mike'], 'Titanic':[5,4,2,5, None],'Batman':[5,5,2,4,3], 'Superman':[4,5,3,None,4], 'Spiderman':[None, 4,1,3,4], 'Avengers':[4,5,3, None, None] }
df = pd.DataFrame(data)
df

,user,Titanic,Batman,Superman,Spiderman,Avengers
0,Tom,5.0,5,4.0,NaN,4.0
1,Sally,4.0,5,5.0,4.0,5.0
2,Vincio,2.0,2,3.0,1.0,3.0
3,Andy,5.0,4,NaN,3.0,NaN
4,Mike,NaN,3,4.0,4.0,NaN


In [390]:
# long-form dataframe
df2 = pd.melt(df, 
               id_vars='user', 
               var_name='movie', 
               value_name='rating')
df2.head()

,user,movie,rating
0,Tom,Titanic,5.0
1,Sally,Titanic,4.0
2,Vincio,Titanic,2.0
3,Andy,Titanic,5.0
4,Mike,Titanic,NaN


In [391]:
#data = pd.read_csv("C:/Users/ahwang/Desktop/Cuny/DATA612/movielens/ratings.csv")

In [392]:
#pd.read_csv("C:/Users/ahwang/Desktop/Cuny/DATA612/movielens/movies.csv")

# Split 80/20 (train/test) for df, by eliminating entire row

In [393]:
train, test = train_test_split(df, test_size = 0.2, random_state = 0)
train

,user,Titanic,Batman,Superman,Spiderman,Avengers
0,Tom,5.0,5,4.0,NaN,4.0
1,Sally,4.0,5,5.0,4.0,5.0
3,Andy,5.0,4,NaN,3.0,NaN
4,Mike,NaN,3,4.0,4.0,NaN


In [394]:
train.sum()[1:].sum()

64.0

In [395]:
test

,user,Titanic,Batman,Superman,Spiderman,Avengers
2,Vincio,2.0,2,3.0,1.0,3.0


# Split 80/20 (train/test) for df2 (converted to long-form), by removing points in row and column 

In [396]:
train2, test2 = train_test_split(df2, test_size = 0.2, random_state = 42)

In [397]:
train2 = pd.pivot_table(train2, index = 'user', columns = 'movie', values = 'rating' ).reset_index()
train2

movie,user,Avengers,Batman,Spiderman,Superman,Titanic
0,Andy,NaN,NaN,3.0,NaN,5.0
1,Mike,NaN,3.0,4.0,4.0,NaN
2,Sally,5.0,5.0,NaN,NaN,4.0
3,Tom,4.0,5.0,NaN,4.0,NaN
4,Vincio,3.0,2.0,1.0,3.0,2.0


In [398]:
test2 = pd.pivot_table(test2, index = 'user', columns = 'movie', values = 'rating' ).reset_index()
test2

movie,user,Batman,Spiderman,Superman,Titanic
0,Andy,4.0,NaN,NaN,NaN
1,Sally,NaN,4.0,5.0,NaN
2,Tom,NaN,NaN,NaN,5.0


# Raw average - df & df2

In [399]:
raw_avg = train.sum()[1:].sum() / train.iloc[:,1:].notnull().sum().sum()

In [400]:
raw_avg2 = train2.sum()[1:].sum() / train2.iloc[:,1:].notnull().sum().sum()

# RMSE for test set - df

In [401]:
error_test = test.iloc[:,1:] - raw_avg
sq_error_test = error_test ** 2 
MSE_test = sq_error_test.sum().sum() / test.iloc[:,1:].notnull().sum().sum()
RMSE_test = np.sqrt(MSE_test)
RMSE_test

2.197978869577938

# RMSE for test set - df2

In [402]:
error_test2 = test2.iloc[:,1:] - raw_avg2
sq_error_test2 = error_test2 ** 2 
MSE_test2 = sq_error_test2.sum().sum() / test2.iloc[:,1:].notnull().sum().sum()
RMSE_test2 = np.sqrt(MSE_test2)
RMSE_test2

1.0625

# RMSE for training set - df

In [403]:
error_train = train.iloc[:,1:] - raw_avg
sq_error_train = error_train ** 2 
MSE_train = sq_error_train.sum().sum() / train.iloc[:,1:].notnull().sum().sum()
RMSE_train = np.sqrt(MSE_train)
RMSE_train

0.6798692684790381

# RMSE for training set - df2

In [404]:
error_train2 = train2.iloc[:,1:] - raw_avg2
sq_error_train2 = error_train2 ** 2 
MSE_train2 = sq_error_train2.sum().sum() / train2.iloc[:,1:].notnull().sum().sum()
RMSE_train2 = np.sqrt(MSE_train2)
RMSE_train2

1.1709371246996996

# Bias Values

In [595]:
#bias_user using df
sum_item_user = train.sum(axis=1)
n_item_user = train.iloc[:,1:].notnull().sum(axis=1)

bias_user = (sum_item_user / n_item_user) - raw_avg
bias_user.index = list(train.user)
bias_user

Tom      0.233333
Sally    0.333333
Andy    -0.266667
Mike    -0.600000
dtype: float64

In [666]:
raw_avg

4.266666666666667

In [677]:
#bias_user using df2
sum_item_user2 = train2.sum(axis=1)
n_item_user2 = train2.iloc[:,1:].notnull().sum(axis=1)

bias_user2 = (sum_item_user2 / n_item_user2) - raw_avg2
bias_user2.index = list(train2.user)
bias_user2

Andy      0.437500
Mike      0.104167
Sally     1.104167
Tom       0.770833
Vincio   -1.362500
dtype: float64

In [676]:
#bias_movie using df
sum_item_movie = train.iloc[:,1:].sum(axis=0)
n_item_movie = train.iloc[:,1:].notnull().sum(axis=0)

bias_movie = (sum_item_movie / n_item_movie) - raw_avg
bias_movie

Titanic      0.400000
Batman      -0.016667
Superman     0.066667
Spiderman   -0.600000
Avengers     0.233333
dtype: float64

In [689]:
bias_user

Tom      0.233333
Sally    0.333333
Andy    -0.266667
Mike    -0.600000
dtype: float64

In [675]:
#bias_movie using df2
sum_item_movie2 = train2.iloc[:,1:].sum(axis=0)
n_item_movie2 = train2.iloc[:,1:].notnull().sum(axis=0)

bias_movie2 = (sum_item_movie2 / n_item_movie2) - raw_avg2
bias_movie2

movie
Avengers     0.437500
Batman       0.187500
Spiderman   -0.895833
Superman     0.104167
Titanic      0.104167
dtype: float64

# baseline predictor

In [678]:
#using df
bias_movie_df = pd.DataFrame(bias_movie).T
base_line_train = pd.DataFrame(columns = bias_movie.index)
for i in train['user']:
    base_line_train = pd.concat([bias_movie_df + bias_user[i] + raw_avg, base_line_train])

In [679]:
#using df2
bias_movie_df2 = pd.DataFrame(bias_movie2).T
base_line_train2 = pd.DataFrame(columns = bias_movie2.index)
for i in train2['user']:
    base_line_train2 = pd.concat([bias_movie_df2 + bias_user2[i] + raw_avg2, base_line_train2])

In [680]:
# Anyway, this removing raw (by df) is not possible since in baseline preidictor, everything will be null..
# Why? because base_line train does not contain vinicio.
base_line_train

,Titanic,Batman,Superman,Spiderman,Avengers
0,4.066667,3.650000,3.733333,3.066667,3.900000
0,4.400000,3.983333,4.066667,3.400000,4.233333
0,5.000000,4.583333,4.666667,4.000000,4.833333
0,4.900000,4.483333,4.566667,3.900000,4.733333


In [688]:
test - base_line_train

,Avengers,Batman,Spiderman,Superman,Titanic,user
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


In [681]:
base_line_train.index = train.index

In [685]:
base_line_train

,Titanic,Batman,Superman,Spiderman,Avengers
0,4.066667,3.650000,3.733333,3.066667,3.900000
1,4.400000,3.983333,4.066667,3.400000,4.233333
3,5.000000,4.583333,4.666667,4.000000,4.833333
4,4.900000,4.483333,4.566667,3.900000,4.733333


In [683]:
# Andy Batman, Sally Spiderman - super man, Tom titanic
test

,user,Titanic,Batman,Superman,Spiderman,Avengers
2,Vincio,2.0,2,3.0,1.0,3.0


In [673]:
error_train2 = train2.iloc[:,1:] - raw_avg2
sq_error_train2 = error_train2 ** 2 
MSE_train2 = sq_error_train2.sum().sum() / train2.iloc[:,1:].notnull().sum().sum()
RMSE_train2 = np.sqrt(MSE_train2)
RMSE_train2

1.1709371246996996

In [674]:
# this is why df is not possible.. deleted df... use df2 only ...!!
error_test2_baseline = test2 - base_line_train2
sq_error_test2_baseline = error_test2_baseline ** 2
MSE_train2_baseline = sq_error_test2_baseline.sum().sum() / test2.iloc[:,1:].notnull().sum().sum()
RMSE_train2_baseline = np.sqrt(MSE_train2_baseline)
RMSE_train2

1.1709371246996996

In [ ]:


RMSE_train2_baseline = np.sqrt(MSE_train2_baseline)
RMSE_train2_baseline

movie,Avengers,Batman,Spiderman,Superman,Titanic,user
0,NaN,5.366944,NaN,NaN,NaN,NaN
1,NaN,NaN,1.604444,1.604444,NaN,NaN
2,NaN,NaN,NaN,NaN,0.871111,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


In [626]:
#base_line_train['user'] = list(train.user)

In [637]:
test.iloc[:,1:]

,Titanic,Batman,Superman,Spiderman,Avengers
2,2.0,2,3.0,1.0,3.0


In [642]:
base_line_train.index = train.index

In [643]:
base_line_train

,Titanic,Batman,Superman,Spiderman,Avengers
0,4.066667,3.650000,3.733333,3.066667,3.900000
1,4.400000,3.983333,4.066667,3.400000,4.233333
3,5.000000,4.583333,4.666667,4.000000,4.833333
4,4.900000,4.483333,4.566667,3.900000,4.733333


In [652]:
test - base_line_train

,Avengers,Batman,Spiderman,Superman,Titanic,user
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
